### Link drive

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


### Import necessary libraries

In [ ]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn import metrics
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import os
from pathlib import Path
import gdown
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential                                    
from keras.preprocessing import text, sequence
from keras.layers import Flatten, Dense  
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow import keras
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Get Data

In [ ]:
%cd "/content/gdrive/MyDrive/"
if not os.path.exists("Youtube_review"):
    os.makedirs("Youtube_review")

/content/gdrive/MyDrive


In [ ]:
%cd "Youtube_review"

/content/gdrive/MyDrive/Youtube_review


In [ ]:
id = "13MsyV3HPqe1sNOMg66CzHbhpMiGrDuLi"
output = "IMDB Dataset.csv"
gdown.download(id = id,output = output, quiet=False)
"""
@InProceedings{maas-EtAl:2011:ACL-HLT2011,
  author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
  title     = {Learning Word Vectors for Sentiment Analysis},
  booktitle = {Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies},
  month     = {June},
  year      = {2011},
  address   = {Portland, Oregon, USA},
  publisher = {Association for Computational Linguistics},
  pages     = {142--150},
  url       = {http://www.aclweb.org/anthology/P11-1015}
}
"""

Downloading...
From: https://drive.google.com/uc?id=13MsyV3HPqe1sNOMg66CzHbhpMiGrDuLi
To: /content/gdrive/MyDrive/Youtube_review/IMDB Dataset.csv
100%|██████████| 66.2M/66.2M [00:00<00:00, 71.2MB/s]


'\n@InProceedings{maas-EtAl:2011:ACL-HLT2011,\n  author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},\n  title     = {Learning Word Vectors for Sentiment Analysis},\n  booktitle = {Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies},\n  month     = {June},\n  year      = {2011},\n  address   = {Portland, Oregon, USA},\n  publisher = {Association for Computational Linguistics},\n  pages     = {142--150},\n  url       = {http://www.aclweb.org/anthology/P11-1015}\n}\n'

In [ ]:
id = "1xzYTdI4GfyFdWW7ZogFFX5tHX93CoUA_"
output = "youtube_data.csv"
gdown.download(id = id,output = output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1xzYTdI4GfyFdWW7ZogFFX5tHX93CoUA_
To: /content/gdrive/MyDrive/Youtube_review/youtube_data.csv
100%|██████████| 304k/304k [00:00<00:00, 46.8MB/s]


'youtube_data.csv'

In [ ]:
for dirname, _, filenames in os.walk(os.getcwd()):
    for filename in filenames:
      print(os.path.join(dirname, filename))

/content/gdrive/MyDrive/Youtube_review/IMDB Dataset.csv
/content/gdrive/MyDrive/Youtube_review/youtube_data.csv
/content/gdrive/MyDrive/Youtube_review/model/best_model.h5


In [ ]:
data_path = "/content/gdrive/MyDrive/Youtube_review/IMDB Dataset.csv"
df = pd.read_csv(data_path)

### Normalize Data

In [ ]:
def one_hot(text):
  if(text == 'positive'):
    return 1
  else:
    return 0
df['sentiment']=df['sentiment'].apply(one_hot)

In [ ]:
words = []
for i in df.review.values:
    words.append(i.split())
# words[:1]

In [ ]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(words)
tokenized_train = tokenizer.texts_to_sequences(words)
X = sequence.pad_sequences(tokenized_train, maxlen = 100, padding='post')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, df.sentiment , test_size = 0.3 , random_state = 0) 

In [ ]:
num_words = len(tokenizer.word_index) + 1
print(num_words)

390932


### Build Model

In [ ]:
if not os.path.exists("model"):
    os.makedirs("model")
# format folder model
for f in os.listdir("./model"):
    os.remove(os.path.join("./model", f))
check_point = ModelCheckpoint("./model/best_model.h5", monitor = 'val_acc', verbose = 1, save_best_only=True)

In [ ]:
model = Sequential()
model.add(Embedding(num_words, 64))
model.add(Bidirectional(LSTM(32))) 
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          25019648  
                                                                 
 bidirectional (Bidirectiona  (None, 64)               24832     
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 25,046,593
Trainable params: 25,046,593
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size= 1024, validation_data = (X_test,y_test), callbacks = [check_point])

Epoch 1/10
35/35 [==============================] - ETA: 0s - loss: 0.6795 - acc: 0.6060
Epoch 1: val_acc improved from -inf to 0.74440, saving model to ./model/best_model.h5
35/35 [==============================] - 30s 725ms/step - loss: 0.6795 - acc: 0.6060 - val_loss: 0.5564 - val_acc: 0.7444
Epoch 2/10
35/35 [==============================] - ETA: 0s - loss: 0.3962 - acc: 0.8281
Epoch 2: val_acc improved from 0.74440 to 0.84713, saving model to ./model/best_model.h5
35/35 [==============================] - 27s 774ms/step - loss: 0.3962 - acc: 0.8281 - val_loss: 0.3569 - val_acc: 0.8471
Epoch 3/10
35/35 [==============================] - ETA: 0s - loss: 0.1615 - acc: 0.9449
Epoch 3: val_acc improved from 0.84713 to 0.85260, saving model to ./model/best_model.h5
35/35 [==============================] - 26s 757ms/step - loss: 0.1615 - acc: 0.9449 - val_loss: 0.3804 - val_acc: 0.8526
Epoch 4/10
35/35 [==============================] - ETA: 0s - loss: 0.0632 - acc: 0.9823
Epoch 4: val_a

In [ ]:
model1 = keras.models.load_model("./model/best_model.h5")

### Testing youtube data crawled before

In [ ]:
df_test_new2 = pd.read_csv("youtube_data.csv",encoding='utf8')

In [ ]:
df_test_new2

,review,Trang,Uyên,Đạt,Gold
0,- 100k more dislikes then likes - “How you han...,0.0,0.0,0.0,0
1,"-""We were instructed not to interfere in any h...",2.0,2.0,2.0,-
2,"-""Why didn’t you guys help fight Thanos?"" -""W...",2.0,2.0,2.0,-
3,-how are you? -gucci gang -oh okay,2.0,2.0,2.0,-
4,-Netflix: “We care about protecting children f...,0.0,0.0,0.0,0
...,...,...,...,...,...
2994,GUCCI GANG IS STILL DOPE IN 2022 YEAHHH ;),1.0,1.0,1.0,1
2995,GUCCI GANG IS THE BEST SONG EVA,1.0,1.0,1.0,1
2996,Gucci Gang lyrics: Gucci Gang.,1.0,1.0,1.0,1
2997,gucci gang still bumpin hard🌐🌐🌐🌐🌐🛸🛸🛸🛸🛸,1.0,1.0,1.0,1


In [ ]:
print(len(df_test_new2))

2999


In [ ]:
df_test_new2 = df_test_new2.drop(df_test_new2.index[df_test_new2['Gold'] == "-"])

In [ ]:
df_test_new2 = df_test_new2.dropna()

In [ ]:
df_test_new2['Gold'] = df_test_new2['Gold'].astype(int)

In [ ]:
df_test_new2

,review,Trang,Uyên,Đạt,Gold
0,- 100k more dislikes then likes - “How you han...,0.0,0.0,0.0,0
4,-Netflix: “We care about protecting children f...,0.0,0.0,0.0,0
5,:35 seconds into this trailer I find myself as...,0.0,0.0,0.0,0
6,:D I’m loving it dude,1.0,1.0,1.0,1
7,:how many drugs do you want in this video? Lil...,0.0,0.0,0.0,0
...,...,...,...,...,...
2994,GUCCI GANG IS STILL DOPE IN 2022 YEAHHH ;),1.0,1.0,1.0,1
2995,GUCCI GANG IS THE BEST SONG EVA,1.0,1.0,1.0,1
2996,Gucci Gang lyrics: Gucci Gang.,1.0,1.0,1.0,1
2997,gucci gang still bumpin hard🌐🌐🌐🌐🌐🛸🛸🛸🛸🛸,1.0,1.0,1.0,1


### Split data

In [ ]:
len_A=len(df_test_new2)//2
X_A=df_test_new2['review'][0:len_A]
y_A=df_test_new2['Gold'][0:len_A]

In [ ]:
X_B=df_test_new2['review'][len_A:]
y_B=df_test_new2['Gold'][len_A:]

### Predict data A

In [ ]:
tokenized_X_A = tokenizer.texts_to_sequences(X_A)
paded_X_A = sequence.pad_sequences(tokenized_X_A, maxlen = 100, padding='post')
y_pred_A = model1.predict(paded_X_A)>0.5
print(accuracy_score(y_A,y_pred_A))

0.5801169590643275


### Train more data by data B

In [ ]:
tokenized_X_B = tokenizer.texts_to_sequences(X_B)
paded_X_B = sequence.pad_sequences(tokenized_X_B, maxlen = 100, padding='post')
X_B1, X_B2, y_B1, y_B2 = train_test_split(paded_X_B, y_B , test_size = 0.3 , random_state = 0) 
model1 = keras.models.load_model("./model/best_model.h5")
check_point = ModelCheckpoint("./model/best_model.h5", monitor = 'val_acc', verbose = 1, save_best_only=True)
history = model1.fit(X_B1, y_B1, epochs=10, batch_size= 1024, validation_data = (X_B2,y_B2), callbacks = [check_point])

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.8845 - acc: 0.5953
Epoch 1: val_acc improved from -inf to 0.57198, saving model to ./model/best_model.h5
1/1 [==============================] - 6s 6s/step - loss: 0.8845 - acc: 0.5953 - val_loss: 0.6983 - val_acc: 0.5720
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.6919 - acc: 0.6120
Epoch 2: val_acc improved from 0.57198 to 0.59922, saving model to ./model/best_model.h5
1/1 [==============================] - 2s 2s/step - loss: 0.6919 - acc: 0.6120 - val_loss: 0.7263 - val_acc: 0.5992
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.6968 - acc: 0.6003
Epoch 3: val_acc improved from 0.59922 to 0.66537, saving model to ./model/best_model.h5
1/1 [==============================] - 2s 2s/step - loss: 0.6968 - acc: 0.6003 - val_loss: 0.6564 - val_acc: 0.6654
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.6251 - acc: 0.6940
Epoch 4: val_acc did not improve from 0.

### Predict data A after train more data B

In [ ]:
model1 = keras.models.load_model("./model/best_model.h5")
y_pred_A = model1.predict(paded_X_A)>0.5
print(accuracy_score(y_A,y_pred_A))

0.7134502923976608


In [ ]:
print(
    f"Classification report for classifier {model1}:\n"
    f"{metrics.classification_report(y_A, y_pred_A)}\n"
)

Classification report for classifier <keras.engine.sequential.Sequential object at 0x7faf0ebfa350>:
              precision    recall  f1-score   support

           0       0.64      0.78      0.70       375
           1       0.79      0.66      0.72       480

    accuracy                           0.71       855
   macro avg       0.72      0.72      0.71       855
weighted avg       0.73      0.71      0.71       855




### Testing with an example

In [ ]:
string = "this is a good video"
tokenized_string = tokenizer.texts_to_sequences([string])
paded_string = sequence.pad_sequences(tokenized_string, maxlen = 100, padding='post')
print( 1 if model1.predict(paded_string)>0.5 else 0)

1


In [ ]:
string1 = 'i love it'
tokenized_string1 = tokenizer.texts_to_sequences([string1])
paded_string1 = sequence.pad_sequences(tokenized_string1, maxlen = 100, padding='post')
print(model1.predict(paded_string1))
print(f"string 1: {string1}")
string2 = "I don't love it"
tokenized_string2 = tokenizer.texts_to_sequences([string2])
paded_string2 = sequence.pad_sequences(tokenized_string2, maxlen = 100, padding='post')
print(model1.predict(paded_string2))
print(f"string 2: {string2}")

[[0.5421666]]
string 1: i love it
[[0.5337641]]
string 2: I don't love it
